In [ ]:
import json
import os
import numpy as np
import tensorflow as tf
import tensorflow.keras.layers as tfl
from tensorflow.keras.preprocessing import image
from keras.applications.mobilenet_v2 import preprocess_input, decode_predictions

In [ ]:
# Load the pre-trained image classification model from tensorflow - this might change
model = tf.keras.applications.MobileNetV2(weights='imagenet', include_top=True)
model.summary()

In [11]:
# Look at the top 2 layers 
nb_layers = len(model.layers)
print(model.layers[nb_layers - 2].name)
print(model.layers[nb_layers - 1].name)

global_average_pooling2d_1
predictions


In [ ]:
# Load the JSON data
with open('train_data.json') as a:
    train_data = json.load(a)
with open('test_data.json') as b:
    test_data = json.load(b)

print(train_data[0]['title'])

In [ ]:
# Define the new model
def sentiment_model(image_shape=(224, 224)):
    ''' Define a tf.keras model for binary classification out of the MobileNetV2 model
    Arguments:
        image_shape -- Image width and height
    Returns:
    Returns:
        tf.keras.model
    '''

    input_shape = image_shape + (3,)
        
    base_model = tf.keras.applications.MobileNetV2(input_shape=input_shape,
                                                   include_top=False, 
                                                   weights='imagenet') 
        
    # freeze the base model by making it non trainable
    base_model.trainable = False 

    # create the input layer (Same as the imageNetv2 input size)
    inputs = tf.keras.Input(shape=input_shape) 
    
    # data preprocessing using the same weights the model was trained on
    x = preprocess_input(x) 
    
    # set training to False to avoid keeping track of statistics in the batch norm layer
    x = base_model(x, training=False) 
    
    # add the new Binary classification layers
    # use global avg pooling to summarize the info in each channel
    x = tfl.GlobalAveragePooling2D()(x) 
    #include dropout with probability of 0.2 to avoid overfitting
    x = tfl.Dropout(0.2)(x)
        
    # create a prediction layer with one neuron (as a classifier only needs one)
    prediction_layer = tfl.Dense(1)
    
    ### END CODE HERE
    
    outputs = prediction_layer(x)
    model = tf.keras.Model(inputs, outputs)
    
    return model
    

In [ ]:
# Initialize variables to count correct predictions
total_games = 0
correct_predictions = 0

In [ ]:
# Iterate over each game in the JSON file
for game in data:
    game_name = game['title']
    sentiment = game['sentiment']
    images = game['screenshots']

    # Initialize variables to count sentiments for the game
    positive_count = 0
    negative_count = 0
    
    for image_path in images:
        img = image.load_img(os.path.join('C:/Users/Beste/Desktop/AAB/images-big', image_path), target_size=(224,224))
        img_array = image.img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array /= 255.
        img_array = preprocess_input(img_array)

        prediction = model.predict(img_array)[0]
    
        predicted_sentiment = 'Positive' if prediction[1] > 0.5 else 'Negative'
        
        # Update sentiment counts
        if predicted_sentiment == 'Positive':
            positive_count += 1
        else:
            negative_count += 1
    
    # Decide on the sentiment for the game
    if positive_count > negative_count:
        final_predicted_sentiment = 'Positive'
    else:
        final_predicted_sentiment = 'Negative'
    
    # Compare predicted sentiment with truth
    if final_predicted_sentiment == sentiment:
        correct_predictions += 1
    # Increment total games counter
    total_games += 1


accuracy_percentage = (correct_predictions / total_games) * 100
print(f"Total games: {total_games}, Correct predictions: {correct_predictions}, Accuracy: {accuracy_percentage:.2f}%")